In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
# important packages
	
import pandas as pd					# data manipulation using dataframes
import numpy as np					# data statistical analysis

import seaborn as sns				# Statistical data visualization
import matplotlib.pyplot as plt		# data visualisation
%matplotlib inline

# Importing modified dataset

In [ ]:
df = pd.read_csv("../input/modified-urbansound8kcsv/modified_URBANSOUND8K.csv", index_col = 0)

In [ ]:
df.head()

# Data Preprocessing

In [ ]:
### Define Features(X) & Target(y)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values.reshape(-1,1)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
print(y.shape)

In [ ]:
print(y)

In [ ]:
### Splitting Dataset ###

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2, random_state = 0)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

In [ ]:
num_label = y.shape[1]

In [ ]:
model = Sequential()
model.add(Input(shape = (40, ))) 
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(num_label, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
## callbacks

es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=10,
                   min_delta=0.001)

cp = ModelCheckpoint('./best_model.h5', 
                     monitor='val_loss', 
                     mode='min',
                     save_best_only=True,
                     verbose=1)

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
num_epochs = 100
num_batch_size = 32

history = model.fit(X_train,
                    y_train,
                    epochs = num_epochs,
                    batch_size = num_batch_size,
                    verbose = 2,
                    validation_split = 0.2,
                    callbacks = [es])

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

In [ ]:
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Model Prediction

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred[0]

In [ ]:
y_pred = y_pred > 0.5

In [ ]:
print(y_pred[0], y_test[0])

# Metrices

## Accuracy

In [ ]:
# Accuracy

score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

## Confusion Matrix

In [ ]:
### Confusion Matrix ###

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True)
plt.show()

## Classification Report

In [ ]:
### Classification report ###

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# Model Testing

In [ ]:
# load original dataframe

raw_df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")

In [ ]:
# choose random audio file

import random
import os

dir = "../input/urbansound8k/"

row = raw_df.sample()

row = row.reset_index(drop=True)

audio_file = row['slice_file_name'][0]
audio_path = dir + 'fold' + str(row['fold'][0]) + '/' + row['slice_file_name'][0]
class_label = row['class'][0]
print(audio_path)

In [ ]:
# import librosa library

import librosa							# package for music and audio analysis
import librosa.display
import IPython.display as ipd			# public api for display tool in ipython

In [ ]:
# feature engineering

def feature_extractor(audio_file):
    data,sample_rate = librosa.load(audio_file, res_type='kaiser_fast')
    mfccs_file = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
    return mfccs_file

mfccs_audio = feature_extractor(audio_path)
mfccs_audio = np.mean(mfccs_audio.T,axis=0)
mfccs_audio = mfccs_audio.reshape(-1,1).T

In [ ]:
print(mfccs_audio.shape)

In [ ]:
# class prediction

predict_label = model.predict(mfccs_audio)
predict_label = (predict_label > 0.5).T
predict_class = labelencoder.inverse_transform(predict_label) 

In [ ]:
# check real and prediction class

print(predict_class, class_label)

!!yayyyy

In [ ]:
# Now listen music od predicted class

ipd.Audio(audio_path)

# Conclusion

Our trained model obtained a Training accuracy of average 98%, Validatioon accuracy of 90% and a Testing accuracy of 88%.
The performance is very good and the model has generalised well, seeming to predict well when tested against new audio data.